# HW_4
### Задача: используя библиотеку kaggle-environments, реализующую функционал взаимодействия между виртуальными агентами в рамках нескольких популярных игр, реализовать самостоятельно несколько агентов и сравнить их в игре «камень-ножницы-бумага».

In [1]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

### 1. Опишем поведение агента, всегда играющего "камень" - это значение 0

In [4]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Writing rock_agent.py


### 2. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [2]:
%%writefile copy_opponent.py
import random

#Example 
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


### 3. Агент, играющий всегда "ножницы":

In [8]:
%%writefile scissors_agent.py
def scissors_agent(observation, configuration):
    return 2

Writing scissors_agent.py


### 4. Агент, всегда играющий "бумагу":

In [9]:
%%writefile paper_agent.py
def paper_agent(observation, configuration):
    return 1

Writing paper_agent.py


### 5. Агент с последовательностью случайных ходов, но с предпочтением одного варианта (60%):

In [10]:
%%writefile biased_random_agent.py
import random

def biased_random_agent(observation, configuration):
    # С вероятностью 60% выбирает "камень", иначе случайный ход
    return 0 if random.random() < 0.6 else random.randrange(1, configuration.signs)

Writing biased_random_agent.py


### 6. Агент, играющий против предыдущего хода оппонента:

In [3]:
%%writefile counter_opponent.py
import random

def counter_opponent(observation, configuration):
    if observation.step == 0:
        # На первом ходе агент выбирает случайный ход, так как у него нет информации о предыдущем ходе оппонента.
        return random.randrange(0, configuration.signs)  # Можно использовать любой ход в диапазоне от 0 до 2.
    else:
        # Получаем предыдущий ход оппонента.
        opponent_previous_move = observation.lastOpponentAction
        
        # Выбираем ход, который побеждает предыдущий ход оппонента.
        # (opponent_previous_move + 1) % 3 выбирает следующий ход в цикле 0 -> 1 -> 2 -> 0.
        return (opponent_previous_move + 1) % configuration.signs

Overwriting counter_opponent.py


### 7. Агент со случайной стратегией:

In [17]:
%%writefile random_agent.py
import random

def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

Writing random_agent.py


### 8. Агент с повторением своих же кодов:

In [4]:
%%writefile repeat_self_agent.py
import random
# Инициализируем переменную last_action, которая будет хранить последний выполненный ход агента.
last_action = 0

def repeat_self_agent(observation, configuration):
    global last_action
    if observation.step == 0:
        # Если это первый шаг, выбираем случайное действие из возможных ходов (от 0 до configuration.signs - 1)
        last_action = random.randrange(0, configuration.signs)
    return last_action


Overwriting repeat_self_agent.py


### 9. Агент, выбирающий случайный ход каждые три раунда:

In [5]:
%%writefile random_every_three_agent.py
import random

def random_every_three_agent(observation, configuration):
    if observation.step % 3 == 0:
        return random.randrange(0, configuration.signs)
    else:
        return observation.lastOpponentAction


Overwriting random_every_three_agent.py


### 10. Агент с предсказанием на основе тенденции оппонента:

In [6]:
%%writefile trend_predictor_agent.py
import random
from collections import deque

last_actions = deque(maxlen=5)

def trend_predictor_agent(observation, configuration):
    if observation.step > 0:
        last_actions.append(observation.lastOpponentAction)
    if len(last_actions) >= 3 and len(set(last_actions)) == 1:
        return (last_actions[-1] + 1) % configuration.signs
    return random.randrange(0, configuration.signs)


Overwriting trend_predictor_agent.py


### 11. Агент с ротацией ходов по фиксированному шаблону (камень, бумага, ножницы):

In [22]:
%%writefile rotate_agent.py

rotation = [0, 1, 2]

def rotate_agent(observation, configuration):
    return rotation[observation.step % 3]


Writing rotate_agent.py


### 12. Агент, следящий за ходами с конца и реагирующий на них:

In [24]:
%%writefile last_move_reactor.py

def last_move_reactor(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % configuration.signs
    else:
        return 0


Overwriting last_move_reactor.py


### 13. Агент, копирующий свой предыдущий ход:

In [7]:
%%writefile mimic_self_agent.py
import random

last_move = 0

def mimic_self_agent(observation, configuration):
    global last_move
    if observation.step > 0:
        return last_move
    else:
        last_move = random.randrange(0, configuration.signs)
        return last_move


Overwriting mimic_self_agent.py


### 14. Агент, чередующийся между случайным и предсказуемым ходом:

In [12]:
%%writefile alternate_strategy_agent.py
import random

def alternate_strategy_agent(observation, configuration):
    if observation.step % 2 == 0:
        return random.randrange(0, configuration.signs)
    else:
        return (observation.step + 1) % configuration.signs


Overwriting alternate_strategy_agent.py


### 15. Агент, который меняет стратегию каждые пять ходов:

In [8]:
%%writefile change_strategy_agent.py
import random

strategies = [0, 1, 2]

def change_strategy_agent(observation, configuration):
    if observation.step % 5 == 0:
        return strategies[observation.step // 5 % len(strategies)]
    return random.randrange(0, configuration.signs)


Overwriting change_strategy_agent.py


### Турнирное тестирование

In [22]:
agents = [
    "rock_agent.py", "scissors_agent.py", "paper_agent.py",
    "biased_random_agent.py", "counter_opponent.py", "repeat_self_agent.py",
    "random_every_three_agent.py", "trend_predictor_agent.py", "rotate_agent.py",
    "last_move_reactor.py", "mimic_self_agent.py", "alternate_strategy_agent.py",
    "change_strategy_agent.py"
]

# Запуск турнира между всеми агентами
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        result = evaluate(
            "rps",
            [agents[i], agents[j]],
            configuration={"episodeSteps": 100}
        )
        print(f"Результат между {agents[i]} и {agents[j]}: {result}")


Результат между rock_agent.py и scissors_agent.py: [[99.0, -99.0]]
Результат между rock_agent.py и paper_agent.py: [[-99.0, 99.0]]
Результат между rock_agent.py и biased_random_agent.py: [[0, 0]]
Результат между rock_agent.py и counter_opponent.py: [[-98.0, 98.0]]
Результат между rock_agent.py и repeat_self_agent.py: [[99.0, -99.0]]
Результат между rock_agent.py и random_every_three_agent.py: [[0, 0]]
Результат между rock_agent.py и trend_predictor_agent.py: [[-98.0, 98.0]]
Результат между rock_agent.py и rotate_agent.py: [[0, 0]]
Результат между rock_agent.py и last_move_reactor.py: [[98.0, -98.0]]
Результат между rock_agent.py и mimic_self_agent.py: [[99.0, -99.0]]
Результат между rock_agent.py и alternate_strategy_agent.py: [[0, 0]]
Результат между rock_agent.py и change_strategy_agent.py: [[0, 0]]
Результат между scissors_agent.py и paper_agent.py: [[99.0, -99.0]]
Результат между scissors_agent.py и biased_random_agent.py: [[-50.0, 50.0]]
Результат между scissors_agent.py и counter

In [9]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

In [17]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-99.0, 99.0]]